In [1]:
import torch

torch.cuda.is_available()
#!/usr/bin/env python3

import sys,os
from elftools.elf.elffile import ELFFile
from elftools.elf.segments import Segment
from capstone import *
from capstone.x86 import *

import os
import json 

import torch
from torch.utils.data import DataLoader, TensorDataset
import sys,os
from sklearn.metrics import precision_recall_fscore_support , accuracy_score
import numpy as np

device = 'cuda:2' if torch.cuda.is_available() else 'cpu'


In [2]:

MAX_TOKEN_LEN = 1024
BATCH_SIZE = 50
epochs = 100

EXPERIMENT_NAME = 'cusTokenizer_WP_35k_ATW_signature'
new_vocab_size = 35000
# disassembly_decimal disassembly_all_number_to_words disassembly_decimal 
data_key = "disassembly_all_number_to_words"


from transformers import BartTokenizer,BertTokenizer, BertForNextSentencePrediction,BertForPreTraining,BertConfig,AutoModelForMaskedLM,get_linear_schedule_with_warmup ,BertForSequenceClassification
import torch
from transformers import PreTrainedTokenizerFast ,AutoModelForPreTraining ,BertGenerationEncoder, BartForConditionalGeneration,BertGenerationDecoder,EncoderDecoderModel
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
tokenizer = PreTrainedTokenizerFast.from_pretrained("./../../models/" + EXPERIMENT_NAME)
print(tokenizer.pad_token) 

[PAD]


In [3]:
def generate_param_string(param_list):


    if len(param_list)==0:
        return 'void'
    else:
        return " ".join(param_list)
def unique_items(list_of_lists):
    # Flatten the list of lists
    print(list_of_lists[3].split())
    flattened_list = [item for sublist in list_of_lists for item in sublist.split()]
    # Convert the flattened list to a set to remove duplicates
    unique_elements = set(flattened_list)
    return list(unique_elements)






DATA_PATH = '/home/raisul/ANALYSED_DATA/tokenization_data_single_functions'

TRAIN_DATA_PATH  ='/home/raisul/ANALYSED_DATA/tokenization_data_single_functions/train/'

TEST_DATA_PATH   = '/home/raisul/ANALYSED_DATA/tokenization_data_single_functions/test/'




train_json_files = [os.path.join(TRAIN_DATA_PATH, f) for f in os.listdir(TRAIN_DATA_PATH) ]

test_json_files = [os.path.join(TEST_DATA_PATH, f) for f in os.listdir(TEST_DATA_PATH) ]





def read_corpus(json_files):

    all = []
    all_params = []
    all_returns = []
    all_signatures = []

    for k, j_file in enumerate(json_files):
        # if k>1000:
        #     break
        try:

            with open(j_file, 'r') as file:
                data = json.load(file)
                
                funct = data[data_key]['input']

                # if len(data['signature']['paramTypes'])<5:
                all.append(funct)
                param_string = generate_param_string( data['signature']['paramTypes'] )
                all_params.append( param_string )
                all_returns.append(  data['signature']['returnType']  )
                all_signatures.append(param_string + " [SEP] "+ data['signature']['returnType'] )
        except Exception as e: 
            print(e)
    return all , all_params , all_returns , all_signatures
    


train_text , train_params , train_returns, train_signatures = read_corpus(train_json_files)

test_text  , test_params  , test_returns , test_signatures = read_corpus(test_json_files)


        
# text = text[0:5000]
print("Functions Count: ",len(train_text), '\n')
example = train_text[10]
text = train_text + test_text


unique_types = unique_items(train_params)
print(len(unique_types), '\n',unique_types)




Functions Count:  80000 

['int', '*struct']
30 
 ['int', '*char', '*int', 'short', '*short', '*uint', '*ulong', '*struct', 'ushort', 'long', '*longlong', 'uchar', 'longlong', '*long', 'void', 'ulong', '*bool', 'double', '*ulonglong', 'uint', 'None', '*uchar', '*float', '*ushort', '*void', '*double', 'char', 'bool', 'float', 'ulonglong']


In [4]:
def tokenize(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=1024, return_tensors="pt")

train_source_encodings = tokenize(train_text)
train_target_encodings = tokenize(train_signatures)


test_source_encodings = tokenize(test_text)
test_target_encodings = tokenize(test_signatures)

# Create a TensorDataset and DataLoader
train_dataset = TensorDataset(train_source_encodings['input_ids'], train_source_encodings['attention_mask'], train_target_encodings['input_ids'])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


test_dataset = TensorDataset(test_source_encodings['input_ids'], test_source_encodings['attention_mask'], test_target_encodings['input_ids'])
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)




In [5]:
from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained("./../../models/"+EXPERIMENT_NAME+"_model.ckpt", forced_bos_token_id=0)
# tok = BartTokenizer.from_pretrained("facebook/bart-large")
# example_english_phrase = "UN Chief Says There Is No <mask> in Syria"
# batch = tok(example_english_phrase, return_tensors="pt")
# generated_ids = model.generate(batch["input_ids"])
# assert tok.batch_decode(generated_ids, skip_special_tokens=True) == [
#     "UN Chief Says There Is No Plan to Stop Chemical Weapons in Syria"
# ]

In [6]:
# from transformers import LongformerModel, LongformerTokenizer

# # Load tokenizer and model
# tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
# model = LongformerModel.from_pretrained('allenai/longformer-base-4096')

# # Example of how to prepare inputs
# inputs = tokenizer("Example of a long input sequence " * 50, return_tensors="pt", max_length=1024, truncation=True)
# output = model(**inputs)

In [7]:
sep_token_id = tokenizer.sep_token_id
pad_token_id = tokenizer.pad_token_id
cls_token_id = tokenizer.cls_token_id



# def align_output(pred, gt):
#     params_pred = []
#     params_gt = []
#     ret_pred = []
#     ret_gt = []
#     for i,gid in enumerate(gt):
#         if gid!= cls_token_id:
            
            
#             aligned_pred.append(pred[i])
#             aligned_gt.append(gid)

            

In [ ]:

model.to(device)


In [ ]:

for epoch in range(1):  # Number of epochs

##########################################
    #eval
    model.eval()
    with torch.no_grad():
        test_all_ground_truth = []
        test_all_prediction = []
        right = 0
        wrong = 0
        for n,test_batch in enumerate(test_loader):
    
            test_input_ids, test_attention_mask, test_labels = [x.to(device) for x in test_batch]
            # print(input_ids.shape , attention_mask.shape , labels.shape )
            test_outputs = model(input_ids=test_input_ids, attention_mask=test_attention_mask)
            test_token_predictions = torch.argmax(test_outputs.logits, axis=-1)
    
            test_labels_list = test_labels.tolist()
            test_token_predictions_list = test_token_predictions.tolist()
            
            for  j,test_label in enumerate(test_labels_list):
                if test_label==test_token_predictions_list[j]:
                    right = right +1
                else:
                    wrong = wrong+ 1
                for i, lid in enumerate(test_label):
                    if lid not in  [cls_token_id , sep_token_id , pad_token_id]:
                        test_all_ground_truth.append(lid)
                        test_all_prediction.append(test_token_predictions_list[j][i])
    
    
    
    
        print("# # #" *10)
        # test_pred_tokens = tokenizer.convert_ids_to_tokens(test_token_predictions[0].tolist())
        # test_gt_tokens = tokenizer.convert_ids_to_tokens(test_labels[0].tolist())
        # print('test_pred_tokens' , test_pred_tokens)
        # print('test_gt_tokens' ,test_gt_tokens)
        print ('epoch: ',epoch , ' n: ', n )
        print(len(test_all_ground_truth) , len(test_all_prediction))
        print(test_all_ground_truth[-10:])
        print(test_all_prediction[-10:])
        test_precision, test_recall, test_f1, _ = precision_recall_fscore_support(test_all_ground_truth,test_all_prediction ,average='weighted')
        print('TEST precision, recall, f1' ,test_precision, test_recall, test_f1)
    
        print(test_labels_list[0])
        print(test_token_predictions_list[0])
        print("right",right, "  wrong : ", wrong)
    
        # Save the model
    


In [ ]:
sep_token_id = tokenizer.sep_token_id
pad_token_id = tokenizer.pad_token_id
cls_token_id = tokenizer.cls_token_id
